In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [5]:
train = train.drop(["Id","Alley","FireplaceQu","PoolQC","Fence","MiscFeature"] , axis = 1)
test = test.drop(["Alley","FireplaceQu","PoolQC","Fence","MiscFeature"] , axis = 1)


In [6]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler



In [7]:
num_pipeline = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())
cat_pipeline = make_pipeline(SimpleImputer(strategy="most_frequent"), OneHotEncoder(handle_unknown="ignore"))

In [8]:
from sklearn.compose import make_column_selector , make_column_transformer
preprocessing = make_column_transformer(
    (num_pipeline , make_column_selector(dtype_include=np.number)),
    (cat_pipeline , make_column_selector(dtype_include=np.object)),
)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


In [9]:
#train_prepared = preprocessing.fit(train)
#test_prepared = preprocessing.fit(test)

In [10]:
X = train.drop(["SalePrice"] , axis=1)
Y = train["SalePrice"]



In [11]:
from sklearn.model_selection import cross_val_score

In [12]:
from sklearn.linear_model import LinearRegression
lin_reg = make_pipeline(preprocessing , LinearRegression())
lin_reg.fit(X , Y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fba989ad390>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                              

In [13]:
from sklearn.metrics import mean_squared_error
predictions = lin_reg.predict(X)
lin_rmse = mean_squared_error(Y , predictions , squared=False)
lin_rmse

20939.666267878747

In [14]:
lin_rmses = -cross_val_score(lin_reg , X , Y , scoring="neg_root_mean_squared_error" , cv=10 )

In [15]:
pd.Series(lin_rmses).describe()

count       10.000000
mean     32263.622087
std      12976.559297
min      22424.234963
25%      23393.176337
50%      26747.005865
75%      38813.347611
max      61919.104297
dtype: float64

In [16]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = make_pipeline(preprocessing , DecisionTreeRegressor(random_state=42))
tree_reg.fit(X , Y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fba989ad390>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                              

In [17]:
from sklearn.metrics import mean_squared_error
predictions = tree_reg.predict(X)
tree_rmse = mean_squared_error(Y , predictions , squared=False)
tree_rmse

0.0

In [18]:

tree_rmses = -cross_val_score(tree_reg , X , Y , scoring="neg_root_mean_squared_error" , cv=10 )

In [19]:
pd.Series(tree_rmses).describe()

count       10.000000
mean     38665.270063
std      11105.188875
min      29069.687734
25%      32996.226293
50%      35687.051972
75%      39411.783576
max      68295.307920
dtype: float64

In [20]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = make_pipeline(preprocessing , RandomForestRegressor(random_state=42))
forest_reg.fit(X , Y)


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fba989ad390>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                              

In [21]:
forest_rmses = -cross_val_score(forest_reg , X , Y , scoring="neg_root_mean_squared_error" , cv=10 )

In [22]:
pd.Series(forest_rmses).describe()

count       10.000000
mean     29031.971238
std       6978.442195
min      22451.832380
25%      24880.554420
50%      25817.398197
75%      30978.836334
max      43311.637855
dtype: float64

In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
full_pipeline = Pipeline(
[
    ("preprocessing" , preprocessing),
    ("random_forest" , RandomForestRegressor(random_state=42)),
])

param_grid = [
    {'random_forest__max_features':[8,10,12]},
]
grid_search = GridSearchCV(full_pipeline , param_grid , cv=5 , scoring='neg_root_mean_squared_error')
grid_search.fit(X,Y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7fba989ad390>),
                                                                        ('pipeline-2',
                                                                         Pipeline(steps=[('simpleimputer',
                                 

In [24]:
grid_search.best_params_

{'random_forest__max_features': 12}

In [25]:
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)

# extra code – these few lines of code just make the DataFrame look nicer
cv_res = cv_res[[
                 "param_random_forest__max_features", "split0_test_score",
                 "split1_test_score", "split2_test_score", "mean_test_score"]]
score_cols = ["split0", "split1", "split2", "mean_test_rmse"]
cv_res.columns = ["max_features"] + score_cols
cv_res[score_cols] = -cv_res[score_cols].round().astype(np.int64)

cv_res.head()

,max_features,split0,split1,split2,mean_test_rmse
2,12,25630,33215,36559,31257
1,10,26197,33961,36142,31647
0,8,26550,33878,37431,32048


In [26]:
final_model = grid_search.best_estimator_

In [27]:
acc = round(final_model.score(X, Y) * 100, 2)
print('Accuracy of the model: {0}%'.format(acc))
Y_pred = final_model.predict(test)

Accuracy of the model: 97.89%


In [28]:
output = pd.DataFrame({
        "Id": test["Id"],
        "SalePrice": Y_pred
    })

output.to_csv('submission.csv', index=False)